In [1]:
import uproot
import awkward as ak

In [2]:
file = uproot.open("root://xcache//store/user/paus/nanosu/A01/QCD_Pt_80to120_TuneCP5_13TeV_pythia8+RunIISummer20UL18MiniAODv2-106X_upgrade2018_realistic_v16_L1v1-v1+MINIAODSIM//011FFC64-48D6-ED41-9A65-E2815CDD94BA.root")

In [3]:
events = file['Events']

In [4]:
Jets = ak.zip(
    {
        "pt": events['Jet_pt'].array(),
        "eta": events['Jet_eta'].array(),
        "phi": events['Jet_phi'].array(),
        "mass": events['Jet_mass'].array(),
        "jetId": events['Jet_jetId'].array(),
    }
)
jetCut = (Jets.pt > 30) & (abs(Jets.eta) < 4.7)
ak4jets = Jets[jetCut]
ht = ak.sum(ak4jets.pt, axis=-1)

In [5]:
trigger = events['HLT_TripleMu_5_3_3_Mass3p8_DCA'].array()
ak.sum(trigger)

1

In [6]:
Cands = ak.zip(
    {
        "pt": events['PFCands_trkPt'].array()[trigger],
        "eta": events['PFCands_trkEta'].array()[trigger],
        "phi": events['PFCands_trkPhi'].array()[trigger],
        "mass": events['PFCands_mass'].array()[trigger],
    },
    with_name="Momentum4D",
)

In [ ]:
cut = (
    (events['PFCands_fromPV'].array()[trigger] > 1) &
    (events['PFCands_trkPt'].array()[trigger] >= 0.7) &
    (abs(events['PFCands_trkEta'].array()[trigger]) <= 2.5) &
    (abs(events['PFCands_dz'].array()[trigger]) < 10) &
    (events['PFCands_dzErr'].array()[trigger] < 0.05)
)

In [ ]:
cut